# Spoofed Speech Detection via Maximum Likelihood Estimation of Gaussian Mixture Models
The goal of synthetic speech detection is to determine whether a speech segment $S$ is natural or synthetic/converted speeach.

This notebook implements a Gaussian mixture model maximum likelihood (GMM-ML) classifier for synthetic (spoofed) speech detection. This approach uses regular mel frequency cepstral coefficients (MFCC) features and gives the best performance on the [ASVspoof 2015 dataset](https://www.idiap.ch/dataset/avspoof) among the standard classifiers (GMM-SV, GMM-UBM, ...). For more background information see: *Hanilçi, Cemal, Tomi Kinnunen, Md Sahidullah, and Aleksandr Sizov. "Classifiers for synthetic speech detection: a comparison." In INTERSPEECH 2015*. The scripts use the Python package [Bob.Bio.SPEAR 2.04](https://pypi.python.org/pypi/bob.bio.spear/2.0.4) for speaker recogntion.

This work is part of the ["DDoS Resilient Emergency Dispatch Center"](https://www.dhs.gov/science-and-technology/news/2015/09/04/dhs-st-awards-university-houston-26m-cyber-security-research) project at the University of Houston, funded by the Department of Homeland Security (DHS).


April 19, 2015

Lorenzo Rossi

(lorenzo **[dot]** rossi **[at]** gmail **[dot]** com)

In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy
import cqcc_ext
import cfcc_ext
from bob.bio.spear import preprocessor, extractor
from bob.bio.gmm import algorithm
from bob.io.base import HDF5File
from bob.learn import em
from sklearn.metrics import classification_report, roc_curve, roc_auc_score

pd.set_option('display.max_rows', 300)

WAV_FOLDER = '/home/wujunting/data/ASVspoof2019/LA/' #'ASV2019dataset/wav/' # Path to folder containing speakers .wav subfolders
LABEL_FOLDER = '/home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_cm_protocols/' #'ASV2019dataset/CM_protocol/' # Path to ground truth csv files

evalu_before = 'ASVspoof2019_LA_eval'
train_before = 'ASVspoof2019_LA_train'

EXT = '.wav'
%matplotlib inline

## Loading the Ground Truth
Load the dataframes (tables) with the labels for the training, development and evaluation (hold out) sets. Each subfolder corresponds to a different speaker. For example, T1 and D4 indicate the subfolders associated to the utterances and spoofed segments of speakers T1 and D4, respectively in training and development sets. Note that number of evaluation samples >> number of development samples >> testing samples.

You can either select the speakers in each set one by one, *e.g.*:
```
train_subfls = ['T1', 'T2']
``` 
will only load segments from speakers T1 and T2 for training,

or use all the available speakers in a certain subset by leaving the list empty, *e.g.*:
```
devel_subfls = [] 
```
will load all the available Dx speaker segments for the development stage. If you are running this notebook for the first time, you may want to start only with 2 or so speakers per set for sake of quick testing. All the scripts may take several hours to run on the full size datsets.

In [6]:
train_subfls = ['all/segment']#, 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T13']  #T13 used instead of T10 for gender balance
devel_subfls = []#, 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']
evalu_subfls = ['wav']#, 'E2', 'E3', 'E4', 'E5', 'E6','E7',  'E8', 'E9', 'E10']
#train = pd.read_csv(LABEL_FOLDER + 'ASVspoof2019.LA.cm.train.21000.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
#train.sort_values(['folder', 'file'], inplace=True)
train = pd.read_csv(LABEL_FOLDER + 'ASVspoof2019.LA.cm.train.trn.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
train.sort_values(['folder', 'file'], inplace=True)
print(train)
devel = pd.read_csv(LABEL_FOLDER + 'ASVspoof2019.LA.cm.dev.trl.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
devel.sort_values(['folder', 'file'], inplace=True)
#print(devel)
evalu = pd.read_csv(LABEL_FOLDER +'ASVspoof2019.LA.cm.eval.rest.txt', sep=' ', header=None, names=['folder','file','default','method','source'])
print(evalu)
evalu.sort_values(['folder', 'file'], inplace=True)

label_2_class = {'bonafide':1, 'spoof':0}

print('training samples:',len(train))
print('development samples:',len(devel))
print('evaluation samples:',len(evalu))

        folder          file default method    source
2580   LA_0079  LA_T_1004644       -    A01     spoof
13980  LA_0079  LA_T_1006715       -    A04     spoof
10180  LA_0079  LA_T_1007408       -    A03     spoof
10181  LA_0079  LA_T_1017386       -    A03     spoof
6380   LA_0079  LA_T_1025525       -    A02     spoof
...        ...           ...     ...    ...       ...
13979  LA_0098  LA_T_9962675       -    A03     spoof
25379  LA_0098  LA_T_9982036       -    A06     spoof
21579  LA_0098  LA_T_9983869       -    A05     spoof
17779  LA_0098  LA_T_9984124       -    A04     spoof
2579   LA_0098  LA_T_9987202       -      -  bonafide

[25380 rows x 5 columns]
        folder          file default method    source
9322   LA_0009  LA_E_1925654       -      -  bonafide
9332   LA_0002  LA_E_2999092       -      -  bonafide
9340   LA_0029  LA_E_9502838       -      -  bonafide
9370   LA_0059  LA_E_3065402       -      -  bonafide
9389   LA_0052  LA_E_4772226       -      -  bonafide
..

## Speech Preprocessing and MFCC Extraction
Silence removal and MFCC feature extraction for training segments. More details about the bob.bio.spear involved libraries at:
https://www.idiap.ch/software/bob/docs/latest/bioidiap/bob.bio.spear/master/implemented.html

You can also skip this stage and load a set of feaures (see **Loading features** cell).

In [7]:
# Parameters
n_ceps = 84 # number of ceptral coefficients (implicit in extractor)
silence_removal_ratio = .1

In [9]:
subfolders = train_subfls
ground_truth = train

# initialize feature matrix
features = []
y_list = []
y = np.zeros((len(ground_truth),))
print("Extracting features for training stage.")

k = 0
start_time = time.clock()

n_subfls = len(subfolders)
for folder in subfolders[0:n_subfls]:
    print(folder, end=", ")
    train_folder = "".join([WAV_FOLDER,train_before,'/'])
    folder = "".join((train_folder,folder,'/'))
    print(folder)
    f_list = os.listdir(folder)
    # print(f_list)
    for f_name in f_list:
        if f_name.endswith(EXT):
        # ground truth
            try: 
                # 根据train中file(路径)字段，提取相应.wav文件的source(spoof or human)字段
                label = ground_truth[ground_truth.file==f_name[0:12]].source.values[0]
                #print(f_name[0:12], label)
            except IndexError:
                continue
            y_list.append(label_2_class[label])
            # silence removal
            # print(vad)
            rate,x = cqcc_ext.read(folder+f_name)
            vad_data = cfcc_ext.vad(x)
            # CFCC extraction 
            cqcc_feature = cqcc_ext.cqcc(vad_data,rate)
            print(cqcc_feature)
            print("?????????????????")
            features.append(cqcc_feature)
            k += 1

y = np.array(y_list)
Xf = np.array(features)
print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")

Extracting features for training stage.
all/segment, /home/wujunting/data/ASVspoof2019/LA/ASVspoof2019_LA_train/all/segment/
[[-5.13428218e+02  4.42899172e+01 -5.18599457e+01 -5.88597324e+01
   6.25586823e+01  7.14289962e-01 -2.39188917e-01 -7.94800541e+00
   6.57427012e-01  1.46846581e+01 -5.47047300e+00  1.34186473e+01
  -3.77283615e+00 -1.63440800e+00  1.84101188e+01 -1.28095939e+00
  -7.91950608e+00  2.48819080e+00  1.37516957e+01  5.31388923e+00
  -6.24497265e+00  1.48507573e+01 -1.16247371e+00 -1.77350826e+00
   6.46787526e+00  1.19741111e+01 -1.46776870e+01  1.61456768e+01
   3.20496050e+00  1.56277254e+00 -4.95554131e-01  1.02158665e+01
  -1.99131549e+00  7.29799536e+00  3.17947244e+00  3.07023725e+00
   1.75553611e+00  1.42031541e+00  4.50227006e+00  9.08729629e+00
  -9.22644017e-01  2.80419088e+00  6.08790916e+00  1.66982977e+00
   3.53617525e+00  4.10804497e+00  1.89957388e+00  5.75425704e+00
   1.81446534e+00  7.08545070e+00  9.36332571e-02  1.44055877e-01
   6.45846370e+00

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # This is added back by InteractiveShellApp.init_path()


[[-7.09384613e+02  1.33844690e+01 -7.81625597e+01 -7.15908918e+01
   2.16134994e+01  3.59384900e+01 -9.93935043e+00  2.49735308e+00
   1.40882192e+01  3.44978386e+01  2.43486046e+01 -9.19257567e+00
  -4.72896837e+00 -1.43868917e-01 -3.00782012e+00  1.17386592e+01
  -6.60440073e+00  1.82313338e+01  4.67959565e+00  1.08697001e+01
   5.04460695e+00  3.81111711e+00 -1.44781046e+00  1.51098629e+01
  -2.84633070e+00  1.05778706e+01  7.67481708e+00  6.11019319e+00
   9.51727000e+00  2.74108947e+01 -1.65033079e+01  6.79630840e+00
   4.36759033e-01  2.64034015e+00  8.57676718e+00  2.10602257e+01
   3.34186223e+00  1.55120214e+01 -7.66516266e+00  1.39403735e+00
  -7.60501245e+00  1.45348853e+01  6.08538767e+00  1.56375951e+01
  -1.26488693e+00  8.80136671e+00 -3.42447674e+00  3.67140059e+00
   7.58207951e-03  9.86842293e+00  2.05612783e+00  4.21537602e+00
  -6.16974888e+00  1.02126039e+01  1.32029740e+01  1.35445893e+01
   6.37163767e+00  1.45862473e+01  2.55737483e+00  1.34727523e+01
  -9.01944

[[-5.73611098e+02  5.51184799e+00 -9.16283711e+01 ...  1.52925242e+01
  -2.14054030e+00  1.30732897e+01]
 [-4.88591391e+02 -4.02302842e+01 -1.01024969e+02 ...  4.69709628e+00
   4.72207972e+00  1.02033035e+01]
 [-5.20091373e+02 -4.40648215e+01 -1.01725894e+02 ...  6.46773066e+00
   3.20885783e+00  5.47241125e+00]
 ...
 [-4.37280073e+02  5.46375837e-01 -2.74780049e+01 ...  6.36991680e+00
   6.52817057e+00  3.50126937e+00]
 [-4.53834626e+02  5.57527775e+01 -6.19415533e+01 ...  7.62755122e+00
   2.09049653e+00  1.44195496e+00]
 [-3.81154768e+02  2.86065845e+01 -5.26226022e+01 ...  8.25577036e+00
   1.04086656e+00  4.96764629e+00]]
?????????????????
[[-476.0858288   107.30568729  -38.53677704 ...    9.48323021
    -2.19715396    3.25935048]
 [-521.20805688   87.85978418   23.54307053 ...    6.62511882
    -1.37811257    4.66550332]
 [-580.83108974   37.65028737   63.64866012 ...    7.99275022
    -4.14056407    3.93240691]
 ...
 [-664.95865303  -28.87063023  -31.55057125 ...    7.94771846


[[-8.64651984e+02  9.39696460e+01 -1.43818887e+01 -1.24894047e+01
  -2.43181426e+01 -1.07002744e+01 -7.28278914e+00  2.73375449e+01
   2.22674079e+01  2.32272612e+01  4.33650299e+00  2.44776195e+01
   4.31737357e+00  1.08863148e+01  1.16916006e+00  5.53154388e+00
  -1.68113193e+01  2.92491756e+00 -9.70759634e-01  1.43588446e+01
   4.14362174e+00  1.65722173e+01 -1.84197819e+00  1.06431456e+01
  -1.04510219e+01  1.50883609e+01  6.44312498e+00  2.61339224e+00
  -1.06314796e+00  1.43026689e+01 -1.71961030e-01  1.29508696e+01
   1.25336617e+01  1.38639543e+01 -1.54285796e+01  1.78071184e+00
   7.93222373e-01  1.03037252e+01 -6.00558342e+00  7.68788860e+00
  -4.65295055e+00  1.02126885e+01 -2.18036680e+00  9.10437084e+00
  -5.85772358e+00  2.63241120e+00 -6.97380772e+00  8.70087167e+00
   1.46734219e-01  1.54988282e+01 -2.12627817e+00  1.26969616e+01
  -4.52525400e+00  9.65720692e+00 -1.56912077e+00  1.48181345e+01
   6.18823618e+00  9.46587887e+00 -9.26361038e+00  8.86597118e+00
   2.71726

...

[[-5.47882677e+02  7.65609930e+01 -2.60794254e+01 -6.26744684e+01
  -4.88791578e+01  3.10202941e+01  4.28183149e+00  2.62669660e+01
  -2.78044766e+00 -6.02939634e+00  1.48410941e+00  9.78773548e+00
  -6.25114195e+00  2.08415986e+01  8.05894872e+00 -1.92672257e+01
   1.71730427e+01  1.66043083e+01 -1.31319966e+01 -3.20988448e+00
   2.97314954e+00  4.13225717e+00  4.38373170e+00  2.50842766e+01
  -6.97478481e+00  1.03727834e+01 -8.30216447e+00  9.85253585e+00
  -3.01702072e+00  6.40719802e+00 -7.56043927e+00  1.99996496e+01
   9.56720937e-01  4.30920197e+00  3.70270013e+00  6.98476164e+00
  -4.28774636e+00  2.63239313e-01  9.27782918e+00  1.12898631e+01
  -9.37377009e+00  1.18491874e+01  8.08144476e+00 -3.00403474e+00
   1.22866756e+01 -8.94087861e+00 -5.79230308e-01  1.95283532e+01
  -4.96502483e+00  6.24111379e+00  1.67067929e+00  1.22008760e+01
  -2.85078057e+00  6.35701957e+00 -6.02371914e+00  1.08890809e+01
   2.36534977e+00  9.66996467e+00 -8.11230040e+00  1.61278246e+01
  -2.35742

[[-4.81949430e+02  1.05118735e+01 -1.46376269e+01 -1.30563749e+00
   1.58886602e+01 -2.30184792e+01 -4.58257707e-01 -1.40538739e+01
   5.10013597e+00  2.46733826e+01  1.97945665e+01 -1.83500009e+01
   1.88825335e+01  2.62708460e-01 -1.43040592e+00 -1.20828023e+01
   7.01127094e-01  1.60369054e+01 -8.65209841e+00  2.27718276e+01
  -2.40343345e+00  1.28440596e+01 -1.45591910e+01  5.49775603e+00
   6.07090980e+00  1.63054161e+00  5.46008826e+00  7.01118958e+00
  -2.90109506e+00  7.52353483e+00 -3.59741614e+00  1.07470444e+01
   2.95781204e+00 -3.36305952e+00  9.03720794e+00  1.22939464e+00
  -5.30942722e+00  2.26005465e+01 -9.83658763e+00  1.93672492e+00
   6.96508490e+00 -6.44725325e-01  1.08055090e+01  5.69167422e+00
  -6.62000490e+00  1.21103451e+01 -9.66212990e+00  7.25321668e+00
   8.46793596e+00  4.48963324e+00 -4.95724510e-01 -1.85873912e-01
   1.11103265e+01 -5.28779909e-01  2.93890690e+00  8.67058979e+00
  -5.57179320e+00  7.32714458e+00 -1.83568959e+00  1.41494965e+01
  -1.91549

[[-3.51790633e+02  8.09585308e+01 -8.16102153e+01 ... -5.71996058e-01
   2.15331211e+00  8.70697386e+00]
 [-3.95015105e+02  9.78328410e+01 -6.10872787e+01 ...  9.38951218e+00
  -2.83252286e+00  4.33262900e+00]
 [-3.77278776e+02  6.34501903e+01 -3.08839047e+01 ...  7.85170208e+00
   6.52708214e-01  4.78801395e+00]
 ...
 [-6.46203477e+02  4.87618626e+01 -7.84411261e+01 ...  6.55201905e+00
   3.35431167e-01  1.10397973e+01]
 [-6.79433282e+02  4.81048604e+01 -4.54862151e+01 ...  8.60185396e+00
   6.46959407e+00  1.50376861e+01]
 [-7.06006931e+02  5.00772329e+01 -4.91688093e+01 ...  1.09381728e+01
   6.97507040e-01  8.23393412e+00]]
?????????????????
[[-8.47205409e+02  5.32403980e+01 -9.51741355e+01 -2.54956758e+01
  -4.15014776e+00  1.01028749e+01  1.16971101e+01  9.45282655e+00
   3.29873487e+00  2.91945811e+01  1.22253486e+00  8.43474186e+00
  -6.00134217e+00  1.86739801e+00 -1.00484951e+01  1.66067738e+01
   1.28222191e+01  1.27092749e+01 -7.19279401e+00  1.25576911e+01
   4.69647071e+0

[[-4.41529333e+02  6.88714760e+01 -5.96734816e+01 ...  9.58611177e+00
   3.74716349e+00  5.99303015e+00]
 [-4.50240731e+02  5.42991310e+01 -5.53744271e+01 ...  1.08616760e+01
  -3.30461515e+00  1.01561012e+01]
 [-4.36260959e+02 -1.12977995e+01 -3.45144382e+01 ...  1.23148345e+01
   3.53396634e+00  5.45433277e+00]
 ...
 [-5.74779597e+02  6.28787135e+01 -9.58061523e+01 ...  1.09713145e+01
  -3.45550374e+00  8.80914311e+00]
 [-7.79510434e+02  1.60128994e+02 -3.41163030e+01 ...  9.45605399e+00
   1.60201447e+00  9.77099269e+00]
 [-7.50947565e+02  9.64169284e+01  9.92962675e+00 ...  5.30299843e+00
  -4.56449891e-01  7.97345442e+00]]
?????????????????
[[-9.33846161e+02  8.18296237e+01  2.38899481e+00 -2.80694132e+01
  -4.05078694e+01 -6.12982961e+00  3.87532255e+00  2.90114231e+01
   1.66736130e+01  2.12578242e+01  2.68027843e+00  1.11758523e+01
  -7.50379215e+00 -1.04628702e+00 -1.20008123e+01  6.77546742e+00
  -5.98361903e-01  1.55309512e+01  7.72270350e+00  2.15791510e+01
   5.12823999e+0

KeyboardInterrupt: 

In [27]:
print(Xf[y==1].shape)

(2000,)


In [69]:
np.vstack(Xf[y==1]).shape

(34771, 84)

In [80]:
ground_truth

,folder,file,default,method,source,z,z_gmm,score_gmm
107206,E2,E200001s,-,spoof,spoof,0,1.0,7.622108
98412,E2,E200002s,-,spoof,spoof,0,1.0,10.544859
102443,E2,E200003h,-,bonafide,bonafide,1,0.0,-2.888099
103712,E2,E200004s,-,spoof,spoof,0,0.0,-2.797664
98984,E2,E200005h,-,bonafide,bonafide,1,1.0,17.829845
...,...,...,...,...,...,...,...,...
105581,E2,E208996h,-,bonafide,bonafide,1,0.0,-3.521843
100661,E2,E208997h,-,bonafide,bonafide,1,0.0,-3.759985
106825,E2,E208998s,-,spoof,spoof,0,1.0,6.707735
102766,E2,E208999h,-,bonafide,bonafide,1,1.0,7.150391


### Saving features

In [6]:
np.save('X_unbalan_1000_seg.npy',Xf)
np.save('y_unbalan_1000_seg.npy',y)
print('Feature and label matrices saved to disk')

Feature and label matrices saved to disk


### Loading features

In [136]:
# Load already extracter features to skip the preprocessing-extraction stage
# Xf = np.load('train_features_10.npy')
# y = np.load('y_10.npy')
Xf = np.load('X9.npy',allow_pickle=True)
y = np.load('y9.npy',allow_pickle=True)

In [227]:
print(Xf.shape)

(1200,)


# GMM - ML Classification

## GMM Training
Train the GMMs for natural and synthetic speach. For documentation on bob.bio k-means and GMM machines see:
https://pythonhosted.org/bob.learn.em/guide.html

You can also skip the training stage and load an already trained GMM model (see cell **Loading GMM Model**).

In [14]:
# Parameters of the GMM machines
n_gaussians_h = 19 # number of Gaussians
n_gaussians_s = 19
max_iterats = 25 # maximum number of iterations

### GMM for natural speech 

In [15]:
# Initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_nat = em.KMeansMachine(n_gaussians_h,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_nat.shape,Xf[y==1].shape)
em.train(kmeansTrainer, kmeans_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
#kmeans_nat.means

# initialize and train GMM machine
gmm_nat = em.GMMMachine(n_gaussians_h,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_nat.means = kmeans_nat.means
em.train(trainer, gmm_nat, np.vstack(Xf[y==1]), max_iterations = max_iterats, convergence_threshold = 1e-5)
#gmm_nat.save(HDF5File('gmm_nat.hdf5', 'w'))
print("Done in:", (time.clock() - start_time)/60, "minutes")
print(gmm_nat)

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


(19, 84) (14240,)
Done in: 0.44532551666682896 minutes


/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  from ipykernel import kernelapp as app


### GMM for synthetic speech

In [10]:
# initialize and train k-means machine: the means will initialize EM algorithm for GMM machine
start_time = time.clock()
kmeans_synt = em.KMeansMachine(n_gaussians_s,n_ceps)
kmeansTrainer = em.KMeansTrainer()
print(kmeans_synt.shape,Xf[y==0].shape)
em.train(kmeansTrainer, kmeans_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)

# initialize and train GMM machine
gmm_synt = em.GMMMachine(n_gaussians_s,n_ceps)
trainer = em.ML_GMMTrainer(True, True, True)
gmm_synt.means = kmeans_synt.means
em.train(trainer, gmm_synt, np.vstack(Xf[y==0]), max_iterations = max_iterats, convergence_threshold = 1e-5)
print("Done in:", (time.clock() - start_time)/60, "minutes")
#gmm_synt.save(HDF5File('gmm_synt.hdf5', 'w'))
print(gmm_synt)

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


(19, 84) (172220,)
Done in: 6.147974450000038 minutes


/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


### Saving GMM model

In [11]:
gmm_nat.save(HDF5File('gmm_nat_ub1000seg.hdf5', 'w'))
gmm_synt.save(HDF5File('gmm_synt_ub1000seg.hdf5', 'w'))

### Loading GMM model

In [3]:
gmm_nat = em.GMMMachine()
gmm_nat.load(HDF5File('gmm_nat_ub1000seg.hdf5', 'r'))
gmm_synt = em.GMMMachine()
gmm_synt.load(HDF5File('gmm_synt_ub1000seg.hdf5','r'))

In [ ]:
np.save('p_gmm_ml_eval_10.npy',llr_score)
np.save('z_gmm_ml_eval_est_10.npy',z_gmm)

## GMM-ML Scoring
Extract the features for the testing data, compute the likelihood ratio test and  compute ROC AUC and estimated EER scores.

In [11]:
status = 'evalu' # 'devel'(= test) OR 'evalu'(= hold out)
start_time = time.clock()
F_FILE = np.array([])

if status == 'devel':
    subfolders = devel_subfls
    ground_truth = devel
elif status == 'evalu':
    subfolders = evalu_subfls
    ground_truth = evalu
n_subfls = len(subfolders)
# initialize score and class arrays
llr_gmm_score = np.zeros(len(ground_truth),)
z_gmm = np.zeros(len(ground_truth),)
print(status)

vad = preprocessor.Energy_Thr(ratio_threshold=.1)
cepstrum = extractor.Cepstral()

k = 0
thr = .5

#speaker = ground_truth[ground_truth.folder==speaker_id]
f_list = list(ground_truth.file)
evalu_folder = "".join([WAV_FOLDER,evalu_before,'/'])
folder = "".join([evalu_folder,evalu_subfls[0],'/'])
#print(f_list)
#print(folder)
for f in f_list:
    if isinstance(f,str):
        f_name = "".join([folder,f,EXT])
    if os.path.exists(f_name):
        rate,x = cqcc_ext.read(f_name)
        # voice activity detection
        vad_data = cfcc_ext.vad(x)
        
        # CFCC extraction 
        cqcc_feature = cqcc_ext.cqcc(vad_data,rate)
        print(cqcc_feature.shape)
        # Log likelihood ratio computation
        print(gmm_nat(cqcc_feature))
        print(gmm_synt(cqcc_feature))
        llr_gmm_score[k] = gmm_nat(cqcc_feature)-gmm_synt(cqcc_feature)
        z_gmm[k] = int(llr_gmm_score[k]>0)
         
        #class_2_label={0:'spoof',1:'bonafide'}
        #if label_2_class[ground_truth.iloc[k].source] != z_gmm[k]:
        #    F_FILE = np.append(F_FILE,ground_truth.iloc[k].file)
            
    k += 1
        
ground_truth['z'] = ground_truth.source.map(lambda x: int(x=='bonafide'))
ground_truth['z_gmm'] = z_gmm
ground_truth['score_gmm'] = llr_gmm_score
#print(roc_auc_score(ground_truth.z, ground_truth.z_gmm))
print(k,"files processed in",(time.clock()-start_time)/60,"minutes.")

/home/wujunting/anaconda3/envs/bob_py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


evalu
(55, 84)
-286.0054898387556
-276.8382206029207
(105, 84)
-277.69236176943116
-259.7503649274413
(98, 84)
-283.61914402150745
-269.8094347550863
(64, 84)
-286.30386459575533
-279.01812949720846
(10, 84)
-282.1520809122583
-273.22535991422046
(10, 84)
-282.2757040049509
-275.7133087542556
(224, 84)
-284.54807943686757
-273.1056777260639
(50, 84)
-286.54862092318336
-276.95143682289046
(25, 84)
-284.1908653127581
-275.35933026936567
(81, 84)
-283.09371261181053
-268.774867447894
(11, 84)
-291.98442398671926
-277.26827549574693
(102, 84)
-286.1346110607776
-276.5744048383604
(34, 84)
-285.36300696013836
-272.26699177260224
(17, 84)
-283.6084595669135
-275.5105273775323
(12, 84)
-292.94885604953055
-284.16233336517706
(7, 84)
-283.94368605097026
-273.6363189999056
(12, 84)
-282.04883132795
-274.07818449042196


KeyboardInterrupt: 

In [11]:
F_FILE

array(['EHUM10001', 'EHUM10003', 'EHUM10004', 'EHUM10005', 'EHUM10006',
       'EHUM10008', 'EHUM10009', 'EHUM10010', 'EHUM10012', 'EHUM10013',
       'EHUM10016', 'EHUM10017', 'EHUM10022', 'EHUM10023', 'EHUM10024',
       'EHUM10025', 'EHUM10026', 'EHUM10027', 'EHUM10028', 'EHUM10029',
       'EHUM10030', 'EHUM10031', 'EHUM10032', 'EHUM10033', 'EHUM10034',
       'EHUM10035', 'EHUM10036', 'EHUM10037', 'EHUM10038', 'EHUM10039',
       'EHUM10040', 'EHUM10041', 'EHUM10042', 'EHUM10043', 'EHUM10044',
       'EHUM10045', 'EHUM10046', 'EHUM10047', 'EHUM10048', 'EHUM10049',
       'EHUM10050', 'EHUM10051', 'EHUM10052', 'EHUM10053', 'EHUM10054',
       'EHUM10055', 'EHUM10057', 'EHUM10058', 'EHUM10059', 'EHUM10060',
       'EHUM10061', 'EHUM10062', 'EHUM10063', 'EHUM10064', 'EHUM10065',
       'EHUM10066', 'EHUM10067', 'EHUM10069', 'EHUM10070', 'EHUM10071',
       'EHUM10072', 'EHUM10073', 'EHUM10074', 'EHUM10075', 'EHUM10078',
       'EHUM10080', 'EHUM10082', 'EHUM10083', 'EHUM10084', 'EHUM

In [17]:
# Performance evaluation

#实际humans个数
humans = sum(ground_truth.z==1)
print("实际humans个数:", humans)

#实际spoofed个数
spoofed = sum(ground_truth.z==0)
print("实际spoofed个数:", spoofed)

#预测humans个数
pre_humans = sum(ground_truth.z_gmm==1)
print("预测humans个数:", pre_humans)

#预测spoofed个数
pre_spoofed = sum(ground_truth.z_gmm==0)
print("预测spoofed个数:", pre_spoofed)

tp = 0
tn = 0
fp = 0
fn = 0
for i in ground_truth.index:
    if ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==1:
        tp = tp + 1
    elif ground_truth.z[i] == 1 and ground_truth.z_gmm[i]==0:
        fn = fn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==0:
        tn = tn + 1
    elif ground_truth.z[i] == 0 and ground_truth.z_gmm[i]==1:
        fp = fp + 1
print("TP_Num:", tp)
print("TN_Num:", tn)
print("FP_Num:", fp)
print("FN_Num:", fn)

tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(fp+tn)
fnr = fn/(tp+fn)
acc = (tp+tn)/(tp+fp+tn+fn)
eer = (fpr*fnr)/(fpr+fnr)
print("TPR:", tpr)
print("FPR:", fpr)
print("TNR:", tnr)
print("FNR:", fnr)
print("ROC:", roc_auc_score(ground_truth.z, ground_truth.z_gmm))
print("Accuracy:", acc)
print("EER:", eer)

# humans = z_gmm[z_dvl==0]
# spoofed = z_gmm[z_dvl==1]
# fnr = 100*(1-(humans<thr).sum()/len(humans))
# fpr = 100*(1-(spoofed>=thr).sum()/len(spoofed))
# print("ROC AUC score:", roc_auc_score(z_dvl,z_gmm))
# print("False negative rate %:", fnr)
# print("False positive rate %:", fpr)
# print("EER %: <=", (fnr+fpr)/2)

实际humans个数: 6355
实际spoofed个数: 50882
预测humans个数: 28009
预测spoofed个数: 29228
TP_Num: 4176
TN_Num: 27049
FP_Num: 23833
FN_Num: 2179
TPR: 0.6571203776553894
FPR: 0.4683974686529618
TNR: 0.5316025313470383
FNR: 0.34287962234461056
ROC: 0.5943614545012139
Accuracy: 0.5455387249506438
EER: 0.19796435637226648


In [62]:
method = {}
for index, row in ground_truth.iterrows():
    if row['z'] != row['z_gmm']:
        if row['method'] in method:
            method[row['method']] = method[row['method']] + 1
        else:
            method[row['method']] = 1
print(method)

{'-': 6757, 'A07': 17, 'A16': 18, 'A10': 28, 'A12': 21, 'A11': 25, 'A13': 6, 'A15': 4, 'A18': 13, 'A17': 14, 'A19': 39, 'A08': 2}


In [47]:
ground_truth

,folder,file,method,source,z,z_gmm,score_gmm
2597,D1,D1_1002598,human,human,1,1.0,1.447147
2598,D1,D1_1002599,human,human,1,1.0,2.336380
2599,D1,D1_1002600,human,human,1,1.0,3.082637
2600,D1,D1_1002601,human,human,1,1.0,3.716948
2601,D1,D1_1002602,human,human,1,1.0,0.914256
...,...,...,...,...,...,...,...
7767,D3,D3_1007768,S1,spoof,0,0.0,-2.641017
7768,D3,D3_1007769,S2,spoof,0,0.0,-0.571958
7769,D3,D3_1007770,S3,spoof,0,0.0,-5.441518
7770,D3,D3_1007771,S4,spoof,0,0.0,-4.762759


### EER computation

Adjust the threshold $thr$ to reduce $FNR-FPR$ for a more accurate estimate of the $EER$.

The Equal Error Rate ($EER$) is the value where the false negative rate ($FNR$) equals the false positive rate ($FPR$). It's an error metric commonly used to characterize biometric systems.

In [47]:
thr = -.115
pz = llr_gmm_score
spoofed = pz[np.array(ground_truth.z)==1]
humans = pz[np.array(ground_truth.z)==0]
fnr = 100*(humans>thr).sum()/len(humans)
fpr = 100*(spoofed<=thr).sum()/len(spoofed)
print("False negative vs positive rates %:", fnr, fpr)
print("FNR - FPR %:", fnr-fpr)
if np.abs(fnr-fpr) <.25:
    print("EER =", (fnr+fpr)/2,"%")
else:
    print("EER ~", (fnr+fpr)/2,"%")

False negative vs positive rates %: 0.3345997002544352 99.53318745441284
FNR - FPR %: -99.19858775415841
EER ~ 49.933893577333635 %
